Импорты

In [6]:
import os
import requests
import re
import csv

from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import drive

In [7]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Константы

In [8]:
CURR_DIR = '/content/drive/MyDrive/lab2'

Функции

Проверка наличия репозиториев датасета

In [9]:
def check_dataset():
    dataset_directory = os.path.join(CURR_DIR, 'dataset')
    if not os.path.exists(dataset_directory):
        os.makedirs(dataset_directory)

def check_repo_dataset(class_name):
    class_folder = os.path.join(CURR_DIR + '\dataset', class_name)
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)
        return class_folder
    else:
        return class_folder

Парсер ссылки на картинку

In [10]:
def parser_url(url):
    pattern = r'img_url=([^&]+)&text='
    match = re.search(pattern, url)

    if match:
        img_url_encoded = match.group(1)
        img_url_decoded = img_url_encoded.replace('%2F', '/').replace('%3A', ':')
        return img_url_decoded
    else:
        print('Ссылка после img_url не найдена в URL')

Вычисления необходимого количества страниц для скачивания (1 страница = 30 картинкам)

In [11]:
def calc_pages(num_images):
    return num_images // 30 + (num_images % 30 > 0) if num_images > 30 else 1

Получение HTML тегов

In [12]:
def get_html_tags(mini_images):
    if mini_images:
        return 'img', 'serp-item__thumb', 'src'
    else:
        return 'a', 'serp-item__link', 'href'

Закачка картинок

In [13]:
def download_image(url, save_path):
    try:
        response = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, stream=True)
        if(response.status_code == 200):
            with open(save_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            return True
        else:
            print(f'Не удалось загрузить изображение: {url}')
            return False
    except Exception as e:
        print(f'Ошибка при загрузке изображения: {url}')
        return False

In [14]:
def download_images(query, num_images, mini_images = False):
    pages = calc_pages(num_images)
    class_folder = check_repo_dataset(query)

    downloaded_count = 0

    base_url = 'https:'

    csv_file_path = class_folder + '_dataset.csv'

    #а вот это чтобы без движков было, грузим странички
    for page in range(0, pages):
        search_url = f'https://yandex.ru/images/search?text={query}&p={page}'

        with open(csv_file_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['date', 'file_name', 'url'])

        #сделал с with для автоматического закрытия соединения
        with requests.get(search_url, headers={'User-Agent':'Mozilla/5.0'}) as response:
            soup = BeautifulSoup(response.text, 'html.parser')

            tag, tag_class, tag_source = get_html_tags(mini_images)

            for a in soup.find_all(tag, class_=tag_class):
                img_url = a[tag_source]
                # получаем полный URL изображения
                if mini_images and not img_url.startswith('http'):
                    img_url = base_url + img_url
                elif img_url.startswith('/images'):
                    img_url = parser_url(img_url)

                #csv_image_filename = image_filename
                image_filename = f'{downloaded_count:04d}.jpg'
                image_path = os.path.join(class_folder, image_filename)
                if(download_image(img_url, image_path)):
                    downloaded_count += 1
                    print(f"Загружено изображений для {query}: {downloaded_count}/{num_images}")

                    with open(csv_file_path, 'a', newline='') as csv_file:
                        csv_writer = csv.writer(csv_file)
                        csv_writer.writerow([datetime.now().strftime('%Y-%m-%d'), image_filename, img_url])

                if(downloaded_count >= num_images):
                    break

main()

In [15]:
def main():
    check_dataset()
    download_images('polar bear', 5, False)
    download_images('brown bear', 5, False)

In [16]:
if __name__ == '__main__':
    main()

Загружено изображений для polar bear: 1/5
Загружено изображений для polar bear: 2/5
Загружено изображений для polar bear: 3/5
Загружено изображений для polar bear: 4/5
Загружено изображений для polar bear: 5/5
Загружено изображений для brown bear: 1/5
Загружено изображений для brown bear: 2/5
Загружено изображений для brown bear: 3/5
Загружено изображений для brown bear: 4/5
Загружено изображений для brown bear: 5/5
